In [15]:
import glob
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib 

def data_pre(df):
    all_null_list = []

    for col in df.columns:
        if df[col].count() == 0:
            all_null_list.append(col)

    df = df.drop(all_null_list, axis=1)

    df = df.drop("種類", axis=1)
    df = df.drop("市区町村コード", axis=1)

    dis = {
        "30分?60分": 45,
        "1H?1H30": 75,
        "1H30?2H": 105,
        "2H?": 120
    }

    df["最寄駅：距離（分）"] = df["最寄駅：距離（分）"].replace(dis).astype(float)

    df["面積（㎡）"] = df["面積（㎡）"].replace("2000㎡以上", 2000).astype(float)

    year_dist = {}
    for i in df["建築年"].value_counts().keys():
        if "平成" in i:
            num = i.split("平成")[1].split("年")[0]
            year = 33 - int(num)
        if "昭和" in i:
            num = i.split("昭和")[1].split("年")[0]
            year = 96 - int(num)
        if "令和" in i:
            num = i.split("令和")[1].split("年")[0]
            year = 3 - int(num)
        if "戦前" in i:
            year = 76
        year_dist[i] = year

    df["建築年"] = df["建築年"].replace(year_dist).astype(float)

    year = {
        "年第１四半期":".25",
        "年第２四半期":".50",
        "年第３四半期":".75",
        "年第４四半期":".99"
    }

    year_list = {}
    for i in df["取引時点"].value_counts().keys():
        for k, j in year.items():
            if k in i:
                year_rep = i.replace(k, j)

        year_list[i] = year_rep

    df["取引時点"] = df["取引時点"].replace(year_list).astype(float)

    for col in ["都道府県名", "市区町村名",  "地区名", "最寄駅：名称", "間取り", "建物の構造", "用途", "今後の利用目的", "都市計画", "改装", "取引の事情等"]:
        df[col] = df[col].astype("category")
    
    return df

In [16]:
# データセットの用意

file_path = '.../dataset/nishika/old_apartment_2020/train/01.csv'

files = glob.glob("/tmp/working/dataset/nishika/old_apartment_2020/train/*")

# trainデータ読み込み
data_list = []
for i, file in enumerate(files):
    data_list.append(pd.read_csv(file, index_col=0))

df = pd.concat(data_list)

df = data_pre(df)

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [17]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae

df_train, df_val = train_test_split(df, test_size=0.2)

col = "取引価格（総額）_log"
train_y = df_train[col]
train_x = df_train.drop(col, axis=1)

val_y = df_val[col]
val_x = df_val.drop(col, axis=1)

trains = lgb.Dataset(train_x, train_y)
valids = lgb.Dataset(val_x, val_y)

params = {
    "objective": "regression",
    "metrics": "mae"
}

model = lgb.train(params, trains, valid_sets=valids, num_boost_round=1000, early_stopping_rounds=100)

/opt/conda/lib/python3.6/site-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[1]	valid_0's l1: 0.25204
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's l1: 0.234834
[3]	valid_0's l1: 0.219746
[4]	valid_0's l1: 0.206368
[5]	valid_0's l1: 0.194557
[6]	valid_0's l1: 0.184227
[7]	valid_0's l1: 0.174807
[8]	valid_0's l1: 0.166611
[9]	valid_0's l1: 0.159332
[10]	valid_0's l1: 0.153015
[11]	valid_0's l1: 0.146995
[12]	valid_0's l1: 0.141738
[13]	valid_0's l1: 0.137263
[14]	valid_0's l1: 0.133089
[15]	valid_0's l1: 0.12936
[16]	valid_0's l1: 0.126098
[17]	valid_0's l1: 0.122984
[18]	valid_0's l1: 0.120371
[19]	valid_0's l1: 0.117908
[20]	valid_0's l1: 0.115693
[21]	valid_0's l1: 0.11359
[22]	valid_0's l1: 0.111761
[23]	valid_0's l1: 0.10987
[24]	valid_0's l1: 0.108168
[25]	valid_0's l1: 0.106664
[26]	valid_0's l1: 0.105315
[27]	valid_0's l1: 0.104126
[28]	valid_0's l1: 0.102944
[29]	valid_0's l1: 0.101853
[30]	valid_0's l1: 0.100813
[31]	valid_0's l1: 0.0998555
[32]	valid_0's l1: 0.0990316
[33]	valid_0's l1: 0.0982399
[34]	valid_0's l1: 0.09

[279]	valid_0's l1: 0.0777386
[280]	valid_0's l1: 0.0777319
[281]	valid_0's l1: 0.0777171
[282]	valid_0's l1: 0.077707
[283]	valid_0's l1: 0.0776908
[284]	valid_0's l1: 0.0776873
[285]	valid_0's l1: 0.0776765
[286]	valid_0's l1: 0.077672
[287]	valid_0's l1: 0.0776617
[288]	valid_0's l1: 0.0776433
[289]	valid_0's l1: 0.0776391
[290]	valid_0's l1: 0.0776323
[291]	valid_0's l1: 0.0776228
[292]	valid_0's l1: 0.0776176
[293]	valid_0's l1: 0.0776156
[294]	valid_0's l1: 0.077611
[295]	valid_0's l1: 0.0776018
[296]	valid_0's l1: 0.0775997
[297]	valid_0's l1: 0.077593
[298]	valid_0's l1: 0.0775902
[299]	valid_0's l1: 0.0775776
[300]	valid_0's l1: 0.0775704
[301]	valid_0's l1: 0.0775652
[302]	valid_0's l1: 0.0775614
[303]	valid_0's l1: 0.0775577
[304]	valid_0's l1: 0.0775485
[305]	valid_0's l1: 0.0775389
[306]	valid_0's l1: 0.0775359
[307]	valid_0's l1: 0.0775343
[308]	valid_0's l1: 0.0775313
[309]	valid_0's l1: 0.0775298
[310]	valid_0's l1: 0.0775197
[311]	valid_0's l1: 0.0775155
[312]	valid_0'

[561]	valid_0's l1: 0.0766662
[562]	valid_0's l1: 0.0766669
[563]	valid_0's l1: 0.0766642
[564]	valid_0's l1: 0.0766635
[565]	valid_0's l1: 0.076665
[566]	valid_0's l1: 0.0766641
[567]	valid_0's l1: 0.0766636
[568]	valid_0's l1: 0.0766613
[569]	valid_0's l1: 0.0766619
[570]	valid_0's l1: 0.0766608
[571]	valid_0's l1: 0.0766561
[572]	valid_0's l1: 0.0766573
[573]	valid_0's l1: 0.0766574
[574]	valid_0's l1: 0.0766567
[575]	valid_0's l1: 0.076655
[576]	valid_0's l1: 0.0766547
[577]	valid_0's l1: 0.0766543
[578]	valid_0's l1: 0.0766539
[579]	valid_0's l1: 0.076641
[580]	valid_0's l1: 0.0766412
[581]	valid_0's l1: 0.0766383
[582]	valid_0's l1: 0.0766369
[583]	valid_0's l1: 0.0766333
[584]	valid_0's l1: 0.0766336
[585]	valid_0's l1: 0.0766372
[586]	valid_0's l1: 0.0766385
[587]	valid_0's l1: 0.0766365
[588]	valid_0's l1: 0.0766367
[589]	valid_0's l1: 0.0766356
[590]	valid_0's l1: 0.0766371
[591]	valid_0's l1: 0.0766324
[592]	valid_0's l1: 0.0766337
[593]	valid_0's l1: 0.0766271
[594]	valid_0

[843]	valid_0's l1: 0.0764258
[844]	valid_0's l1: 0.0764263
[845]	valid_0's l1: 0.0764255
[846]	valid_0's l1: 0.0764251
[847]	valid_0's l1: 0.0764237
[848]	valid_0's l1: 0.076421
[849]	valid_0's l1: 0.0764203
[850]	valid_0's l1: 0.0764206
[851]	valid_0's l1: 0.0764208
[852]	valid_0's l1: 0.0764202
[853]	valid_0's l1: 0.0764204
[854]	valid_0's l1: 0.0764198
[855]	valid_0's l1: 0.0764162
[856]	valid_0's l1: 0.0764172
[857]	valid_0's l1: 0.0764173
[858]	valid_0's l1: 0.0764181
[859]	valid_0's l1: 0.076416
[860]	valid_0's l1: 0.0764142
[861]	valid_0's l1: 0.0764103
[862]	valid_0's l1: 0.0764109
[863]	valid_0's l1: 0.076411
[864]	valid_0's l1: 0.0764107
[865]	valid_0's l1: 0.0764116
[866]	valid_0's l1: 0.0764121
[867]	valid_0's l1: 0.0764122
[868]	valid_0's l1: 0.0764139
[869]	valid_0's l1: 0.0764146
[870]	valid_0's l1: 0.076416
[871]	valid_0's l1: 0.0764167
[872]	valid_0's l1: 0.0764173
[873]	valid_0's l1: 0.0764171
[874]	valid_0's l1: 0.0764151
[875]	valid_0's l1: 0.076415
[876]	valid_0's

In [19]:
vals = model.predict(val_x)
mae(vals, val_y)

0.07636201905743618

In [21]:
df_test = pd.read_csv("/tmp/working/dataset/nishika/old_apartment_2020/test.csv", index_col=0)
df_test = data_pre(df_test)

In [23]:
predict = model.predict(df_test)
df_test["取引価格（総額）_log"] = predict

In [24]:
df_test[["取引価格（総額）_log"]].to_csv("submit_test.csv")

In [26]:
pd.DataFrame(model.feature_importance(), index=val_x.columns, columns=["importance"]).sort_values("importance", ascending=False)

,importance
地区名,8205
最寄駅：名称,4980
建築年,3644
取引時点,3480
面積（㎡）,2220
最寄駅：距離（分）,2113
市区町村名,2039
間取り,928
改装,564
容積率（％）,504
